Notebook 1: Trabajas en una compañía en el departamento de soporte TI. Ya llevas varios años y te das cuenta de que el trabajo es repetitivo y los usuarios suelen cometer los mismo errores. Con el tiempo el trabajo se va volviendo muy fácil y estás buscando como hacerlo más interesante de acuerdo a tus habilidades.

Has pensado en utilizar tus conocimientos avanzados en Python y las sofisticadas herrarmientas de Inteligencia Artifical de Google para automatizar tu trabajo! En especial la plataforma de tickets (Trello).

Objetivo: Crear una solución que tome los videos de los problemas de los usuarios, cree, resuma, etiquete y gestione targetas en Trello (Herramienta de tickets de desarrollo y soporte) automáticamente.

Instalamos dependencias

In [ ]:
# Podemos explicar para que se instala esto?
!pip install google-api-python-client

In [ ]:
# Instalamos paquetes para conectarnos a los servicios de Google
!pip install google-auth
!pip install google-auth-oauthlib

In [ ]:
# Instalamos Vertex AI (Plataforma de desarrollo de soluciones de AI Generativa).
# Con esta plataforma podremos acceder a los differentes modelos de Gemini de Google y a otros servicios de Agentes y Chat.
!pip install vertexai

In [ ]:
# Instalarémos un paquete para que Langchain interactue con Google Vertex AI
!pip install langchain-core
!pip install -qU langchain-google-vertexai



#### Mencionar de donde se saca la service account y su paso a paso
Importamos las librerías de Google que instalamos previamente, y configuramos las credenciales de Google para autenticarnos con Google Cloud Platform a través de una cuenta de servicio.


In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Path to your service account key file
SERVICE_ACCOUNT_FILE = "prj-uc-genai-labs-8859d31dca67.json"

# Define the scopes
SCOPES = ["https://www.googleapis.com/auth/cloud-platform"]

# Authenticate and construct service
credentials = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)

Importamos Vertex AI, y desde Vertex AI obtenemos los constructores de las clases GenerativeModel y Part, que nos permitiran interactuar fácilmente con la API y diferentes tipos de formatos con los que podremos interactuar con el modelo (Texto, Imágenes, Audio, etc.)

In [ ]:
import vertexai
from vertexai.generative_models import GenerativeModel, Part

def generate_video_description(project_id, location, model_name, video_file_uri, prompt):
    vertexai.init(project=project_id, location=location)

    model = GenerativeModel(model_name)

    video_file = Part.from_uri(video_file_uri, mime_type="video/mp4")

    contents = [video_file, prompt]

    response = model.generate_content(contents)
    return response.text

# Configuramos los parametros de la API para que funcione con nuestro proyecto de Google Cloud
project_id = "prj-uc-genai-labs"
location = "us-central1"

# Configuramos los parametros del modelo y el tipo de input que le entregarémos.
model_name = "gemini-1.5-pro-preview-0409"
video_file_uri = "gs://workshop_pablo_24/0424.mp4"
prompt = """
  Provide a description of the video.
  The description should also contain anything important which people say in the video.
"""

description = generate_video_description(project_id, location, model_name, video_file_uri, prompt)

# Pequeño Debug para ver que nos está devolviendo la API
print(description)

Ahora que tenemos configurado Google Cloud, Vertex AI, el Modelo y sus Parámetros, querémos interactuar con la API de trello para que Gemini pueda gestionar los tickets. Para esto debemos

1. Crear una cuenta de Trello en https://www.trello.com
2. Creamos un Power App, una especie de conector entre un tablero de trello y su API
3. Generámos y copiamos la API Key para interactuar con la plataforma sin credenciales

In [ ]:
# Esta función podrá ser llamada por el modelo utilizando xxxxxxx
def create_trello_card(name, due, start):
    import requests
    import json

    url = "https://api.trello.com/1/cards"

    headers = {"Accept": "application/json"}

    query = {
        "idList": "662a0837e895ddb3063a6277", # Reemplazar por valores propios
        "key": "28c3d510ab4b942cf31f2993d4059942", # Reemplazar por valores propios
        "token": "ATTAf9894d9fcfa1a44dd9653a65f141f58e7503c196515909dd2eea97fe41b3ed43D5A5A316", # Reemplazar por valores propios
        "name": name,
        "due": due,
        "start": start,
    }

    response = requests.request("POST", url, headers=headers, params=query)

    print(
        json.dumps(
            json.loads(response.text), sort_keys=True, indent=4, separators=(",", ": ")
        )
    )


Creamos el modelo Pydantic #QUE NOS PERMITIRÁ..............

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional
from datetime import datetime

class CreateTrelloCard(BaseModel):
    """
    Function that calls the Trello API to create a new card.
    
    """
    name: str = Field(..., description="Name of the card")
    due: Optional[datetime] = Field(...,description="Name of the card")
    start: Optional[datetime] = Field(...,description="Name of the card")

Establecemos una connexión entre Langchain y el Chat de Vertex AI

In [21]:
from langchain_google_vertexai import ChatVertexAI

In [ ]:
tools = [CreateTrelloCard]
llm = ChatVertexAI(
    model="gemini-1.5-pro-preview-0409",
    credentials=credentials,
    project_id="prj-uc-genai-labs",
    location="us-central1",
)


llm_with_tools = llm.bind_tools(tools)